# **Chapter 7 - Ensemble Learning and Random Forests**
- We all know that If we have a Question and we ask it's Answer from General Public and Accumulate Their Results most of the Times It would be better answer than and Experts Answer. This is known as **Wisdom of Crowd**.
- **Machine Learning** Also Functions and Behaves Similar to This if You Train A *model* that is *Expert* in a Certain Task and Someone Else Creates *1000* Relatively **Dumb Models** and then Culminate there Skills into one than There is a High Possibility His Model Overtaking Your Model in Every Aspect in the Language of Machine Learning it is known as **Ensemble Learning**.

## What We are Going to Learn
- Voting Classifier
- Bagging and Pasting
    - Bagging and Pasting in Scikit-Learn
    - Out-of-Bag Evaluation
- Random Patches and Random Subspaces
- Random Forests
    - Extra-Trees
    - Feature Importance
- Boosting
    - AdaBoost
    - Gradient Boosting
- Stacking Ensemble

# Voting Classifiers
- In this Ensemble We Train Multiple Classifiers and the One Category Which Gets the Most Number of Votes is finally Predicted.
- There are Two Types of Voting Classifiers.
    - Hard Voting Classifier - Takes **Predicted Class** as Inputs
    - Soft Voting Classifier - Takes **Possibilities of Every Class** as Inputs

In [1]:
import pandas as pd
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.25)
X_train = X[:2000]
y_train = y[:2000]
X_test = X[2000:]
y_test = y[2000:]

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[("lr", log_clf), ("rnd", rnd_clf), ("svc", svm_clf)],
    voting="hard"
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rnd', RandomForestClassifier()),
                             ('svc', SVC(probability=True))])

In [3]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.86675
RandomForestClassifier 0.934875
SVC 0.944125
VotingClassifier 0.942375


### **Soft Voting Classifier** - Most of the Times Better than Hard Voting Classifiers

In [4]:
voting_clf_soft = VotingClassifier(
    estimators=[("lr", log_clf), ("rnd", rnd_clf), ("svc", svm_clf)],
    voting="soft"
)
voting_clf_soft.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rnd', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [5]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf_soft):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.86675
RandomForestClassifier 0.93475
SVC 0.944125
VotingClassifier 0.93975


# Bagging and Pasting
- There are Two ways to get a diverse set of *Classifiers* so They don't have same Weak Points :-
    - Training Multiple Type of Classifiers.
    - Training Same Classifiers on Random Subsets of the Same Dataset.
        - When Sampling is *Performed with replacement* it is known as **Bagging**.
        - When Sampling is *Performed without replacement* it is known as **Pasting**.
- Once all the Classifiers are Trained Predictions can be Made Just By Aggregating The Predictors Votes. - Soft Votes

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.94

If you Want to Try Our Pasting Just set **bootstrap=False**.

## Out-of-Bag Evaluation
When Sampling with Bagging Classifier only **67%** of the Instances are **Sampled** and **37%** of the Instances are **Not Sampled** at All So We can Use then as Our Testing set and There Results are Quite Amazing and Accurate and this is known as **Out-of-Bag Evaluation**.

In [7]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1,
    oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.939

In [8]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9395

Now let's See Probability of Every Instance using oob_decision_function_

In [9]:
bag_clf.oob_decision_function_

array([[0.99164927, 0.00835073],
       [0.76260504, 0.23739496],
       [0.91810345, 0.08189655],
       ...,
       [0.84      , 0.16      ],
       [0.09433962, 0.90566038],
       [0.0230608 , 0.9769392 ]])

As You can see that the Results are Almost Equal to the Actual Accuracy.

# Random Patches and Random Subspaces
We all know that Bagging Classifiers Support Sampling Well and we Can Control it with two Hyperparameters:-
- **max_features** - Determines the maximum number of features each base classifier is allowed to use when fitting on a subset of the data.
- **bootstrap_features** - Specifies whether features are sampled with replacement when fitting each base classifier.

# Random Forests
- Ensemble of Decision Trees Trained using Bagging Classifier.

In [10]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=1)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.93925

With Few Exception this Class Has all of the Hyperparameters of Decision Trees and Bagging Classifier.

## Extra-Trees
Exactly Same as Random Forest but it also puts Random Threshholds for each feature which makes it Extreamly Random.

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
ext_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=1)
ext_clf.fit(X_train, y_train)
y_pred = ext_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.92725

## Feature Importance

In [12]:
ext_clf.feature_importances_

array([0.41551428, 0.58448572])

In [13]:
rnd_clf.feature_importances_

array([0.41839679, 0.58160321])

# Boosting
The Ensemble which can convert many Weak Learners into a Strong Learner is known as Boosting Ensemble.
The Whole Idea of Boosting is to train Predictors Sequentially each trying to correct its Predicessor mainly there are two types of Boosting Methods :-
- AdaBoost (Adaptive Boosting)
- Gradient Boosting

## AdaBoost (Adaptive Boosting)
One way to Correct a Predictor from its Predicessor is to pay more attention on the Instances that were underfitted.
To Implement this Methode **AdaBoost** increases the weights of the underfitted instances thus resulting in a way better Ensemble than the normal random Ensemble.

In [14]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train, y_train)
y_pred = ada_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.92375

## Gradient Boosting
It Works just Like AdaBoost but Instead of Increasing Weights of the underfitted instances it Trains the next Predictor on the **Residual Errors** made by the Previous Predictor we can Understand it better with its implementation in Python.

In [21]:
tree_reg_1 = DecisionTreeClassifier(max_depth=2)
tree_reg_1.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

In [22]:
y_train2 = y_train - tree_reg_1.predict(X_train)
tree_reg_2 = DecisionTreeClassifier(max_depth=2)
tree_reg_2.fit(X_train, y_train2)

DecisionTreeClassifier(max_depth=2)

In [23]:
y_train3 = y_train - tree_reg_2.predict(X_train)
tree_reg_3 = DecisionTreeClassifier(max_depth=2)
tree_reg_3.fit(X_train, y_train3)

DecisionTreeClassifier(max_depth=2)

In [25]:
y_pred = tree_reg_3.predict(X_test)
accuracy_score(y_test, y_pred)

0.9015